In [1]:
import numpy as np

In [2]:
from decode_utils import decode, get_percent_correct_from_proba, get_fa, get_dprime
nrImages = 1320

In [ ]:

def create_i1_i2(model_features_):
    lb = ['bear','elephant','person','car','dog','apple','chair','plane','bird','zebra']
    labels = np.repeat(lb, 132,axis=0)
    i1 = np.zeros((nrImages,20), dtype=float) 
    i2 = np.zeros((20, nrImages,len(lb)), dtype=float) 
    dp_m = np.zeros((nrImages,20), dtype=float) 
    i1[:]=np.nan 
    i2[:]=np.nan 
    for j in range(20): 
        print('nrBS: '+str(j)) 
        p = decode(model_features_,labels, seed=j, nrfolds=3) 
        pc = get_percent_correct_from_proba(p,labels, np.array(lb)) 
        i1[:,j] = np.nanmean(pc, axis=1) 
        i2[j] = pc
    return i1, i2

In [4]:
def select_neurons(ev, selected_rates):  
    print(ev.shape, selected_rates.shape)  
    selected_rates = np.array(selected_rates[:, :ev.shape[0]])  
    best_neurons = selected_rates[:,ev > np.nanpercentile(ev, 80, axis=0)]  
    worst_neurons = selected_rates[:,ev < np.nanpercentile(ev, 20, axis=0)]  
    return best_neurons, worst_neurons

In [6]:
def load_model_features(model):
    path_map = {
        "alexnet_relu": "model_features/alexnet_features.npy",
        "resnet": "model_features/resnet50_features.npy",
        "resnet18": "model_features/resnet18_features.npy",
        "resnet18_ssl": "model_features/resnet18_ssl_features.npy",
        "resnet18_robust": "model_features/resnet18_robust_features.npy",
        "resnet_ssl": "model_features/resnet50_ssl_features.npy",
        "resnet_swsl": "model_features/resnet50_swsl_features.npy",
        "resnetSIN": "model_features/resnetSIN_features.npy",
        "resnet152_2": "model_features/resnet152_features.npy",
        "resnet101": "model_features/resnet101_features.npy",
        "resnet101_ssl": "model_features/resnet101_ssl_features.npy",
        "resnet_robust_eps1": "model_features/resnet50_eps1_robust_features.npy",
        "resnet_robust_eps3": "model_features/resnet50_eps3_robust_features.npy",
        "inception": "model_features/inceptionv3_features.npy",
        "inceptionv1": "model_features/inceptionv1_features.npy",
        "vit": "model_features/vit_features.npy",
        "vit_ssl": "model_features/vit_ssl_features.npy",
        "vgg16": "model_features/vgg16_features.npy",
        "vgg16_robust": "model_features/vgg16_robust_features.npy",
        "densenet": "model_features/densenet101_features.npy",
        "densenet169": "model_features/densenet169_features.npy",
        "densenet161": "model_features/densenet161_features.npy",
        "densenet161_robust": "model_features/densenet161_robust_features.npy",
        "densenet121": "model_features/densenet121_features.npy",
        "convnext": "model_features/convnext_features.npy",
        "convnext_ssl": "model_features/convnext_ssl_features.npy",
        "mobilenet": "model_features/mobilenet_features.npy",
        "mobilenet_v2": "model_features/mobilenet_v2_features.npy",
        "mobilenet_robust": "model_features/mobilenet_robust_features.npy",
        "squeezenet": "model_features/squeezenet_features.npy",
        "cornetS_t0": "model_features/CORnet-S_t0_features.npy",
        "cornetS_reg": "model_features/cornetS_regularized_features.npy",
        "cornetRT_t4": "model_features/CORnet-RT_t4_features.npy",
        "nasnet": "model_features/nasnet_features.npy",
        "pnasnet": "model_features/pnasnet_features.npy",
        "swin": "model_features/swin_features.npy",
        "swin_ssl": "model_features/swin_ssl_features.npy",
        "shufflenet": "model_features/shufflenet_features.npy",
        "shufflenet_robust": "model_features/shufflenet_robust_features.npy",
    }
    if model not in path_map:
        raise ValueError("Model not found")
    features = np.load(path_map[model])
    return features.reshape((1320, -1))

In [8]:
good_neurons_models_monkeys = {}
for model in ["resnet"]:
    features = load_model_features(model)
    good_neurons_models_monkeys[model] = {}
    for monkey in ["magneto", "nano"]:
        good_neurons_models_monkeys[model][monkey] = {}
        ev_per_neuron = np.load(f'./results_predictions/monkey2model/{monkey}/ev_{model}.npy')
        best_neurons, worst_neurons = select_neurons(ev_per_neuron, features)
        good_neurons_models_monkeys[model][monkey]["good"] = best_neurons
        good_neurons_models_monkeys[model][monkey]["bad"] = worst_neurons


(100352,) (1320, 100352)
(100352,) (1320, 100352)


In [10]:
good_neurons = good_neurons_models_monkeys[model][monkey]["good"]

proj = np.random.randn(good_neurons.shape[1], 813)
good_neurons = np.dot(good_neurons, proj)

In [ ]:
%%capture --no-display

rng = np.random.default_rng(seed=111)
# sample units in the good units from 10 to 800
model = "resnet"
for monkey in ["magneto", "nano"]:
    for sample_size in range(200, 300, 20):
        i1s_good = []
        for r in range(5):
            indices = rng.choice(good_neurons.shape[1], sample_size, replace=False)
            sampled_good_neurons = good_neurons[:, indices]

            i1_good, _ = create_i1_i2(sampled_good_neurons.T)
            i1s_good.append(i1_good)

        np.save(f'./results_behavior/{model}/{monkey}/i1_good_{sample_size}.npy', i1s_good)


In [14]:
%%capture --no-display
for monkey in ["magneto", "nano"]:
    bad_neurons = good_neurons_models_monkeys[model][monkey]["bad"]
        
    proj = np.random.randn(bad_neurons.shape[1], 813)
    bad_neurons = np.dot(bad_neurons, proj)

    i1_bad, _ = create_i1_i2(bad_neurons.T)

    np.save(f'./results_behavior/{model}/{monkey}/i1_bad.npy', i1_bad)
